In [ ]:
# to do

\ lire tous les dfs & concat le tt
\ lister cutoff, couper data, train, val, test 
\ 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# list_cutoff = ['2017-08-16', '2017-07-31', '2017-07-15', '2017-06-29', '2017-06-13']

list_cutoff = ['2017-08-16']

In [22]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales/data/'

cutoff = '2017-07-31'

In [23]:
df_sales_cutoff = pd.read_csv(path + f'fe/cutoff/{cutoff}/sales_cutoff.csv', sep=';').drop(['id'], axis=1)
#df_enc = pd.read_csv(path + 'fe/cutoff/{}/encoding_agg.csv'.format(cutoff), sep=';').drop(['id', 'bu', 'famid'], axis=1)
#df_lag = pd.read_csv(path + 'fe/cutoff/{}/lag_recurisve.csv'.format(cutoff), sep=';').drop('sales', axis=1)
#df_rol = pd.read_csv(path + 'fe/cutoff/{}/rol_recurisve.csv'.format(cutoff), sep=';').drop('sales', axis=1)
df_features = pd.read_csv(path + f'fe/cutoff/{cutoff}/df_festures.csv', sep=';').drop('sales', axis=1)

df_date_hol_oil = pd.read_csv(path + 'fe/date_holidays_oil.csv', sep=';')
df_store_details = pd.read_csv(path + 'fe/stores_details_transactions.csv', sep=';')

df_sales_cutoff['date'] = pd.to_datetime(df_sales_cutoff['date'])
df_features['date'] = pd.to_datetime(df_features['date'])
df_date_hol_oil['date'] = pd.to_datetime(df_date_hol_oil['date'])
df_store_details['date'] = pd.to_datetime(df_store_details['date'])

assert df_sales_cutoff.groupby(['date', 'ts_id']).size().max() == 1
assert df_features.groupby(['date', 'ts_id']).size().max() == 1
assert df_date_hol_oil.groupby(['date']).size().max() == 1
assert df_store_details.groupby(['date', 'bu']).size().max() == 1

In [24]:
df_sales_cutoff.ts_id.nunique()

1729

In [25]:
df_sales_cutoff.shape, df_features.shape

((2450500, 6), (2272527, 154))

In [26]:
df_sales_cutoff.ts_id.nunique(), df_features.ts_id.nunique()

(1729, 1729)

In [27]:
df_w = pd.merge(df_sales_cutoff, df_features, how='left', on=['date', 'ts_id'])
assert df_w.shape[0] == df_sales_cutoff.shape[0]
assert df_w.groupby(['date', 'ts_id']).size().max() == 1

In [28]:
df_ww = pd.merge(df_w, df_date_hol_oil, how='left', on=['date'])
assert df_ww.shape[0] == df_w.shape[0]
assert df_ww.groupby(['date', 'ts_id']).size().max() == 1

In [29]:
df_www = pd.merge(df_ww, df_store_details, how='left', on=['date', 'bu'])
assert df_www.shape[0] == df_ww.shape[0]
assert df_www.groupby(['date', 'ts_id']).size().max() == 1

In [30]:
df_www.columns

Index(['ts_id', 'date', 'bu', 'famid', 'sales', 'onprom', 'lag_sales_1',
       'lag_sales_2', 'lag_sales_3', 'lag_sales_4',
       ...
       'hol_before_2', 'hol_before_3', 'hol_after_1', 'prixoil',
       'transactions', 'isclosed', 'typeid', 'cityid', 'stateid', 'cluster'],
      dtype='object', length=180)

In [31]:
df_www.shape, df_features.ts_id.nunique()

((2450500, 180), 1729)

In [32]:
df_www.to_csv('data/fe/cutoff/{}/final_frame.csv'.format(cutoff), index=False, sep=';')

In [33]:
df_www.head()

,ts_id,date,bu,famid,sales,onprom,lag_sales_1,lag_sales_2,lag_sales_3,lag_sales_4,...,hol_before_2,hol_before_3,hol_after_1,prixoil,transactions,isclosed,typeid,cityid,stateid,cluster
0,1_0,2013-01-02,1,0,2.0,0,NaN,NaN,NaN,NaN,...,0,0,0,93.14,2111.0,0,0,0,0,13
1,1_0,2013-01-03,1,0,3.0,0,NaN,NaN,NaN,NaN,...,0,0,0,92.97,1833.0,0,0,0,0,13
2,1_0,2013-01-04,1,0,3.0,0,NaN,NaN,NaN,NaN,...,0,0,0,93.12,1863.0,0,0,0,0,13
3,1_0,2013-01-05,1,0,5.0,0,NaN,NaN,NaN,NaN,...,0,0,0,93.12,1509.0,0,0,0,0,13
4,1_0,2013-01-06,1,0,2.0,0,NaN,NaN,NaN,NaN,...,0,0,0,93.12,520.0,0,0,0,0,13
